In [1]:
import matplotlib.pyplot as plt
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
import tensorflow as tf
import numpy as np
import glob
import os
import sys  
import cv2
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn import preprocessing
import tensorflow as tf
import scipy

from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


In [2]:
print(tf.__version__)
data_path="npz_data/ucf_classes/*"
img_size=(224,224,3)
frame_number=20
num_class=3



1.13.1


In [3]:
def load_data(data_path):
    data=np.empty([1,frame_number,img_size[0],img_size[1],3])
    label=np.empty([1,num_class])
    file_list=glob.glob(data_path)
    data_list=[d for d in file_list if "data" in d.split("/")[-1]]
    label_list=[l for l in file_list if "label" in l.split("/")[-1]]
#     for index in range(len(data_list)):
#         d=np.load(data_list[index])["arr_0"]
#         l=np.load(label_list[index])["arr_0"]
#         data=np.append(data,d,axis=0)
#         data=np.append(label,l,axis=0)
#     data=data[1:]
#     label=label[1:]
    for index in range(len(data_list)):
        data=np.load(data_list[index])["arr_0"]
        label=np.load(label_list[index])["arr_0"]
    return data,label

data,label=load_data(data_path)

print(data.shape)

#try to upsize the data
sample=np.empty([len(data),frame_number,img_size[0],img_size[1],3])

print(sample.shape)

for frameCubeNum in range(len(data)):
    frameCube = data[frameCubeNum,...]
    frame_stack=np.empty([frame_number,img_size[0],img_size[1],3])
#     print("size of frame cube {}".format(frame_stack.shape))

    for imgcubeNum in range(len(frameCube)):
        imgcube = frameCube[imgcubeNum,...]
        img_stack=np.empty([img_size[0],img_size[1],3])
#         print("size of imgcube cube {}".format(img_stack.shape))

        for imglayerNum in range(3):
            imglayer = imgcube[...,imglayerNum]
            img_sm = cv2.resize(imglayer, (img_size[0],img_size[1]), interpolation=cv2.INTER_CUBIC)
#             print("size of imglayer {}".format(imglayer.shape))
            img_stack[...,imglayerNum] = img_sm
        
        frame_stack[imgcubeNum,...]=img_stack
#         frame_stack=np.append(frame_stack,img_stack,axis=0)
#         print(frameCube.shape)
        
    sample[frameCubeNum,...]=frame_stack
    
    
    
data = sample

print(data.shape)
print(label.shape)

(150, 20, 56, 56, 3)
(150, 20, 224, 224, 3)
(150, 20, 224, 224, 3)
(150, 3)


In [4]:
label.shape[0]

150

In [5]:
print(data)

[[[[[0.5646808  0.61427401 0.60667083]
    [0.56745425 0.61782707 0.60997608]
    [0.57990411 0.63367077 0.62490485]
    ...
    [0.82827053 0.86356464 0.86748621]
    [0.82688802 0.86218214 0.86610371]
    [0.82658931 0.86188343 0.86580499]]

   [[0.56999331 0.61932063 0.61042415]
    [0.57232307 0.62241862 0.61330845]
    [0.58265817 0.63609949 0.62622164]
    ...
    [0.82827053 0.86356464 0.86748621]
    [0.82688802 0.86218214 0.86610371]
    [0.82658931 0.86188343 0.86580499]]

   [[0.59652358 0.64469186 0.63011326]
    [0.59664208 0.64552901 0.63088703]
    [0.59643685 0.64846048 0.63369808]
    ...
    [0.82836141 0.86365553 0.8675771 ]
    [0.82696267 0.86225679 0.86617836]
    [0.82666089 0.86195501 0.86587658]]

   ...

   [[0.50643006 0.5770183  0.63192026]
    [0.50619837 0.57678661 0.63168857]
    [0.50512605 0.57571429 0.63061625]
    ...
    [0.38383498 0.42493868 0.47158773]
    [0.36250803 0.40031631 0.44765662]
    [0.35797774 0.39503924 0.4425289 ]]

   [[0.50686772 

In [6]:
# train_x = data[:int(label.shape[0]*0.8),...]
# train_y = label[:int(label.shape[0]*0.8),...]
# valid_x = data[int(label.shape[0]*0.8):int(label.shape[0]*0.9),...]
# valid_y = label[int(label.shape[0]*0.8):int(label.shape[0]*0.9),...]
# test_x = data[int(label.shape[0]*0.9):,...]
# test_y = label[int(label.shape[0]*0.9):,...]

train_x, valid_x, train_y,valid_y =  train_test_split(data, label, test_size=0.2, random_state=42)
test_x, valid_x, test_y,valid_y =  train_test_split(valid_x, valid_y, test_size=0.5, random_state=42)



In [7]:
print(test_y.shape)


(15, 3)


In [8]:
import tensorflow_hub as hub


W0510 11:44:37.885994 140708951783168 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [9]:
# learning_rate = 1e-6
X = tf.placeholder(tf.float32,[None,frame_number,224,224,3])
labels = tf.placeholder(tf.float32,[None,num_class])


module = hub.Module("https://tfhub.dev/deepmind/i3d-kinetics-600/1",trainable=False)
outputs = module(X)

dense1 = tf.layers.dense(inputs=outputs, units=128)
dense2 = tf.layers.dense(inputs=dense1, units=64)

logits = tf.layers.dense(inputs=dense2, units=num_class)

prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=labels))
optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(loss_op)
        
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
  

Instructions for updating:
Colocations handled automatically by placer.


W0510 11:44:38.270461 140708951783168 deprecation.py:323] From /home/jetherng/keras/lib/python3.5/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0510 11:44:39.609128 140708951783168 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Use keras.layers.dense instead.


W0510 11:44:39.682957 140708951783168 deprecation.py:323] From <ipython-input-9-a8f528be2fa0>:9: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



W0510 11:44:39.710869 140708951783168 deprecation.py:323] From <ipython-input-9-a8f528be2fa0>:18: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [10]:
batch_size = 2
num_steps = int(train_x.shape[0]/batch_size)
display_step = 2

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(3):
        print("epoch {}".format(epoch))

        for step in range(0, num_steps):
            batch_x, batch_y = train_x[step:(step+1)*batch_size,...],train_y[step:(step+1)*batch_size,...]

                # Run optimization op (backprop)
            sess.run(train_op, feed_dict={X: batch_x, labels: batch_y})
            if step % display_step == 0 or step == 1:
                    # Calculate batch loss and accuracy
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                         labels: batch_y})
                print("Step " + str(step) + ", Minibatch Loss= " + \
                          "{:.4f}".format(loss) + ", Training Accuracy= " + \
                          "{:.3f}".format(acc))

        print("Optimization Finished fpr epoch {}\n".format(epoch))

        print("validation Accuracy:", \
            sess.run(accuracy, feed_dict={X: valid_x,
                                              labels:valid_y}))
        

    print("Testing Accuracy:", \
            sess.run(accuracy, feed_dict={X: test_x,
                                              labels:test_y}))    
 







epoch 0
Step 0, Minibatch Loss= 0.0208, Training Accuracy= 1.000
Step 1, Minibatch Loss= 0.0003, Training Accuracy= 1.000
Step 2, Minibatch Loss= 8.4990, Training Accuracy= 0.750
Step 4, Minibatch Loss= 4.1325, Training Accuracy= 0.500
Step 6, Minibatch Loss= 1.9345, Training Accuracy= 0.625
Step 8, Minibatch Loss= 3.9643, Training Accuracy= 0.700
Step 10, Minibatch Loss= 4.9295, Training Accuracy= 0.667
Step 12, Minibatch Loss= 1.4632, Training Accuracy= 0.786
Step 14, Minibatch Loss= 0.6201, Training Accuracy= 0.938
Step 16, Minibatch Loss= 3.2577, Training Accuracy= 0.722
Step 18, Minibatch Loss= 3.7852, Training Accuracy= 0.750
Step 20, Minibatch Loss= 2.3340, Training Accuracy= 0.773
Step 22, Minibatch Loss= 1.1888, Training Accuracy= 0.833
Step 24, Minibatch Loss= 1.4188, Training Accuracy= 0.846
Step 26, Minibatch Loss= 1.4063, Training Accuracy= 0.786
Step 28, Minibatch Loss= 1.3885, Training Accuracy= 0.867
Step 30, Minibatch Loss= 1.1863, Training Accuracy= 0.844
Step 32, Min

In [11]:
for i in range(3):
    print(i)

0
1
2


In [12]:
a = np.zeros ((5,5))
b = np.ones ((5,5))
c = 2*np.ones((5,5))

In [13]:
d = np.empty([5,5,3])

In [14]:
d.shape

(5, 5, 3)